## Optimal routing from stores having specific medicines available to the required address


In [1]:
from arcgis.gis import GIS
my_gis = GIS('home')

In [2]:
import arcgis
from arcgis.gis import GIS
import pandas as pd
import datetime
import getpass
from IPython.display import HTML
from IPython.display import FileLink, FileLinks

from arcgis import geocoding
from arcgis.features import Feature, FeatureSet
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [3]:
#customer data
customers_csv = "home/customers.csv"

# Read the csv file and convert the data to feature set
customers_df = pd.read_csv(customers_csv)
customers_sdf = pd.DataFrame.spatial.from_df(customers_df, "Address")
customers_fset = customers_sdf.spatial.to_featureset()

In [4]:
customers_sdf.head()

,Address,SHAPE
0,"DADAR AVANTHI CHS, Vijay Nagar Building, 10, S...","{""x"": 72.84650241600008, ""y"": 19.0283986920000..."
1,"802, Old Prabhadevi Rd, Prabhadevi, Mumbai, Ma...","{""x"": 72.82391804000008, ""y"": 19.0134605920000..."
2,"1, Dr Baba Saheb Ambedkar Rd, near Premier Cin...","{""x"": 72.84235605600003, ""y"": 19.0099955950000..."
3,GD Ambedkar Marg Parel Tank Road Kalachowki Am...,"{""x"": 72.84682154500007, ""y"": 18.9968172060000..."
4,"Kalyandas Wadi, Shop No. 15, Dr Baba Saheb Amb...","{""x"": 72.84465630200003, ""y"": 19.0137659430000..."


In [5]:
#medical stores data
stores_csv = "home/stores.csv"

# Read the csv file and convert the data to feature set
stores_df = pd.read_csv(stores_csv)
stores_sdf = pd.DataFrame.spatial.from_xy(stores_df, "Longitude", "Latitude")
stores_sdf = stores_sdf.drop(axis=1, labels=["Longitude", "Latitude"])
stores_fset = stores_sdf.spatial.to_featureset()
stores_sdf

,Name,Metformin,SHAPE
0,Sheth Medical & General Store,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
1,Ashok Medical & General Store,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
2,Jai Hind Medical Stores,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.9..."
3,Kalpana Medical & General Store,not available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
4,Ashapura Chemist,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
5,Roshani Medical & General Store,not available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
6,Sai Medico,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."


In [6]:
# Create a map instance to visualize inputs in map
map_view_inputs = my_gis.map('Mumbai, Maharashtra, IND', zoomlevel=9)
map_view_inputs

MapView(layout=Layout(height='400px', width='100%'))

In [7]:
# Visualize order and depot locations with symbology
map_view_inputs.draw(customers_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_inputs.draw(stores_fset, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})

In [8]:
#routes of all stores
testroutes_csv = "home/testroutes.csv"

# Read the csv file
testroutes_df = pd.read_csv(testroutes_csv, parse_dates=["EarliestStartTime", "LatestStartTime"], date_parser=pd.to_datetime)
testroutes_df["EarliestStartTime"] = testroutes_df["EarliestStartTime"].astype("int64") / 10 ** 6
testroutes_df["LatestStartTime"] = testroutes_df["LatestStartTime"].astype("int64") / 10 ** 6
testroutes_fset = arcgis.features.FeatureSet.from_dataframe(testroutes_df)
route_dummy = testroutes_df.set_index('Stores')
testroutes_df

,ObjectID,Stores,Name,StartDepotName,EndDepotName,EarliestStartTime,LatestStartTime,Capacities,CostPerUnitTime,MaxOrderCount,MaxTotalTime,AssignmentRule
0,1,Sheth Medical & General Store,Route1,Sheth Medical & General Store,Sheth Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
1,2,Ashok Medical & General Store,Route2,Ashok Medical & General Store,Ashok Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
2,3,Jai Hind Medical Stores,Route3,Jai Hind Medical Stores,Jai Hind Medical Stores,1.637741e+12,1.637748e+12,10,20,10,360,1
3,4,Kalpana Medical & General Store,Route4,Kalpana Medical & General Store,Kalpana Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
4,5,Ashapura Chemist,Route5,Ashapura Chemist,Ashapura Chemist,1.637741e+12,1.637748e+12,10,20,10,360,1
5,6,Roshani Medical & General Store,Route6,Roshani Medical & General Store,Roshani Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
6,7,Sai Medico,Route7,Sai Medico,Sai Medico,1.637741e+12,1.637748e+12,10,20,10,360,1


In [9]:
#extract those stores where Metformin medicines are available
stores_med_df = stores_df.loc[stores_df['Metformin']=='available']
stores_med_sdf = pd.DataFrame.spatial.from_xy(stores_med_df, "Longitude", "Latitude")
stores_med_sdf = stores_med_sdf.drop(axis=1, labels=["Longitude", "Latitude"])
stores_med_fset = stores_med_sdf.spatial.to_featureset()
stores_med_sdf

/opt/conda/lib/python3.7/site-packages/arcgis/features/geo/_io/fileops.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SHAPE'] = GeoArray(ags_geom)


,Name,Metformin,SHAPE
0,Sheth Medical & General Store,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
1,Ashok Medical & General Store,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
2,Jai Hind Medical Stores,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.9..."
4,Ashapura Chemist,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."
6,Sai Medico,available,"{""spatialReference"": {""wkid"": 4326}, ""x"": 72.8..."


In [10]:
#download the dataset of medical stores where Metformin medicines are available
stores_med_df.to_csv('home/stores_med.csv', index=False)    #to download the csv file

FileLinks('home')

home/
  .nb_auth_file
  routes.csv
  depots.csv
  Routes_fset.json
  orders.csv
  customers.csv
  stores.csv
  testroutes.csv
  routes_med.json
  stores_med.csv
  routes_med.csv
  Routes_details.csv
  Stops_details.csv
home/pittsburgh_restaurants.gdb/
  a00000001.gdbindexes
  a00000001.gdbtable
  a00000001.gdbtablx
  a00000001.TablesByName.atx
  a00000002.gdbtable
  a00000002.gdbtablx
  a00000003.gdbindexes
  a00000003.gdbtable
  a00000003.gdbtablx
  a00000004.CatItemsByPhysicalName.atx
  a00000004.CatItemsByType.atx
  a00000004.FDO_UUID.atx
  a00000004.gdbindexes
  a00000004.gdbtable
  a00000004.gdbtablx
  a00000004.spx
  a00000005.CatItemTypesByName.atx
  a00000005.CatItemTypesByParentTypeID.atx
  a00000005.CatItemTypesByUUID.atx
  a00000005.gdbindexes
  a00000005.gdbtable
  a00000005.gdbtablx
  a00000006.CatRelsByDestinationID.atx
  a00000006.CatRelsByOriginID.atx
  a00000006.CatRelsByType.atx
  a00000006.FDO_UUID.atx
  a00000006.gdbindexes
  a00000006.gdbtable
  a00000006.gdbtablx
  a00000007.CatRelTypesByBackwardLabel.atx
  a00000007.CatRelTypesByDestItemTypeID.atx
  a00000007.CatRelTypesByForwardLabel.atx
  a00000007.CatRelTypesByName.atx
  a00000007.CatRelTypesByOriginItemTypeID.atx
  a00000007.CatRelTypesByUUID.atx
  a00000007.gdbindexes
  a00000007.gdbtable
  a00000007.gdbtablx
  gdb
  timestamps
  _gdb.0b787b2bb5bf48a.9.8.sr.lock
  a00000004.horizon
  a00000009.gdbtable
  a00000009.gdbtablx
  a00000009.gdbindexes
  a00000009.horizon
  a00000009.spx
  a00000001.freelist
  a0000000b.gdbtable
  a0000000b.gdbtablx
  a0000000b.gdbindexes
  a0000000b.horizon
  a0000000b.spx
  a0000000c.gdbtable
  a0000000c.gdbtablx
  a0000000c.gdbindexes
  a0000000c.horizon
  a00000004.freelist
  a00000006.freelist
  a0000000c.spx
  a0000000d.gdbtable
  a0000000d.gdbtablx
  a0000000d.gdbindexes
  a0000000d.horizon
  a0000000d.spx

In [11]:
#stores' list where Metformin medicines are available
list_of_stores = stores_med_df['Name'].to_list()
list_of_stores

['Sheth Medical & General Store',
 'Ashok Medical & General Store',
 'Jai Hind Medical Stores',
 'Ashapura Chemist',
 'Sai Medico']

In [12]:
#stores where Metformin medicines are not available
for index,row in route_dummy.iterrows():
  if index not in list_of_stores:
    print(str(index)  + ' is not having required medicines')
  else:
    pass

Kalpana Medical & General Store is not having required medicines
Roshani Medical & General Store is not having required medicines


In [13]:
routes_updt = route_dummy[route_dummy.index.isin(list_of_stores)]
routes_df1 = routes_updt.reset_index()
routes_med_df = routes_df1.drop(['Stores'], axis = 1)
routes_med_df    #modified routes dataset where Metformin medicines are available 

,ObjectID,Name,StartDepotName,EndDepotName,EarliestStartTime,LatestStartTime,Capacities,CostPerUnitTime,MaxOrderCount,MaxTotalTime,AssignmentRule
0,1,Route1,Sheth Medical & General Store,Sheth Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
1,2,Route2,Ashok Medical & General Store,Ashok Medical & General Store,1.637741e+12,1.637748e+12,10,20,10,360,1
2,3,Route3,Jai Hind Medical Stores,Jai Hind Medical Stores,1.637741e+12,1.637748e+12,10,20,10,360,1
3,5,Route5,Ashapura Chemist,Ashapura Chemist,1.637741e+12,1.637748e+12,10,20,10,360,1
4,7,Route7,Sai Medico,Sai Medico,1.637741e+12,1.637748e+12,10,20,10,360,1


In [14]:
#download the modified route file
routes_med_df.to_csv('home/routes_med.csv', index=False)    #to download the csv file

FileLinks ('home')

home/
  .nb_auth_file
  routes.csv
  depots.csv
  Routes_fset.json
  orders.csv
  customers.csv
  stores.csv
  testroutes.csv
  routes_med.json
  stores_med.csv
  routes_med.csv
  Routes_details.csv
  Stops_details.csv
home/pittsburgh_restaurants.gdb/
  a00000001.gdbindexes
  a00000001.gdbtable
  a00000001.gdbtablx
  a00000001.TablesByName.atx
  a00000002.gdbtable
  a00000002.gdbtablx
  a00000003.gdbindexes
  a00000003.gdbtable
  a00000003.gdbtablx
  a00000004.CatItemsByPhysicalName.atx
  a00000004.CatItemsByType.atx
  a00000004.FDO_UUID.atx
  a00000004.gdbindexes
  a00000004.gdbtable
  a00000004.gdbtablx
  a00000004.spx
  a00000005.CatItemTypesByName.atx
  a00000005.CatItemTypesByParentTypeID.atx
  a00000005.CatItemTypesByUUID.atx
  a00000005.gdbindexes
  a00000005.gdbtable
  a00000005.gdbtablx
  a00000006.CatRelsByDestinationID.atx
  a00000006.CatRelsByOriginID.atx
  a00000006.CatRelsByType.atx
  a00000006.FDO_UUID.atx
  a00000006.gdbindexes
  a00000006.gdbtable
  a00000006.gdbtablx
  a00000007.CatRelTypesByBackwardLabel.atx
  a00000007.CatRelTypesByDestItemTypeID.atx
  a00000007.CatRelTypesByForwardLabel.atx
  a00000007.CatRelTypesByName.atx
  a00000007.CatRelTypesByOriginItemTypeID.atx
  a00000007.CatRelTypesByUUID.atx
  a00000007.gdbindexes
  a00000007.gdbtable
  a00000007.gdbtablx
  gdb
  timestamps
  _gdb.0b787b2bb5bf48a.9.8.sr.lock
  a00000004.horizon
  a00000009.gdbtable
  a00000009.gdbtablx
  a00000009.gdbindexes
  a00000009.horizon
  a00000009.spx
  a00000001.freelist
  a0000000b.gdbtable
  a0000000b.gdbtablx
  a0000000b.gdbindexes
  a0000000b.horizon
  a0000000b.spx
  a0000000c.gdbtable
  a0000000c.gdbtablx
  a0000000c.gdbindexes
  a0000000c.horizon
  a00000004.freelist
  a00000006.freelist
  a0000000c.spx
  a0000000d.gdbtable
  a0000000d.gdbtablx
  a0000000d.gdbindexes
  a0000000d.horizon
  a0000000d.spx

In [15]:
#convert the modified route file to json file online and import it here
import json
with open("home/routes_med.json") as f:
    df1 = json.load(f)
routes_string = json.dumps(df1)    
routes_fset5 = FeatureSet.from_json(routes_string)
routes_fset5

<FeatureSet> 5 features

In [16]:
routes_string

'{"features": [{"attributes": {"Name": "Route1", "StartDepotName": "Sheth Medical & General Store", "EndDepotName": "Sheth Medical & General Store", "EarliestStartTime": "11/24/2021 8:00", "LatestStartTime": "11/24/2021 10:00", "Capacities": 10, "CostPerUnitTime": 20, "MaxOrderCount": 10, "MaxTotalTime": 360, "AssignmentRule": 1}}, {"attributes": {"Name": "Route2", "StartDepotName": "Ashok Medical & General Store", "EndDepotName": "Ashok Medical & General Store", "EarliestStartTime": "11/24/2021 8:00", "LatestStartTime": "11/24/2021 10:00", "Capacities": 10, "CostPerUnitTime": 20, "MaxOrderCount": 10, "MaxTotalTime": 360, "AssignmentRule": 1}}, {"attributes": {"Name": "Route3", "StartDepotName": "Jai Hind Medical Stores", "EndDepotName": "Jai Hind Medical Stores", "EarliestStartTime": "11/24/2021 8:00", "LatestStartTime": "11/24/2021 10:00", "Capacities": 10, "CostPerUnitTime": 20, "MaxOrderCount": 10, "MaxTotalTime": 360, "AssignmentRule": 1}}, {"attributes": {"Name": "Route5", "Start

In [17]:
%%time
today = datetime.datetime.now()
from arcgis.network.analysis import solve_vehicle_routing_problem
results = solve_vehicle_routing_problem(orders= customers_fset,
                                        depots = stores_med_fset,
                                        routes = routes_fset5, 
                                        save_route_data='true',
                                        populate_directions='true',
                                        travel_mode="Driving Time",
                                        default_date=today)

print('Analysis succeeded? {}'.format(results.solve_succeeded))

WARNING 030194: Data values longer than 500 characters for field [Routes:StartDepotName] are truncated.
WARNING 030194: Data values longer than 500 characters for field [Routes:EndDepotName] are truncated.
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Private Roads" "Through Traffic Prohibited" "Avoid Gates").


Analysis succeeded? True
CPU times: user 248 ms, sys: 12.9 ms, total: 261 ms
Wall time: 12.7 s


In [18]:
# Display the output routes in a pandas dataframe.
out_routes_df = results.out_routes.sdf
out_routes_df[['Name','OrderCount','StartTime','EndTime','TotalCost','TotalDistance','TotalTime','TotalTravelTime','StartTimeUTC','EndTimeUTC']]

,Name,OrderCount,StartTime,EndTime,TotalCost,TotalDistance,TotalTime,TotalTravelTime,StartTimeUTC,EndTimeUTC
0,Route1,7,2021-11-24 08:00:00,2021-11-24 09:05:51.737999916,1317.246164,14.209776,65.862308,65.862308,2021-11-24 02:30:00,2021-11-24 03:35:51.737999916
1,Route2,8,2021-11-24 08:00:00,2021-11-24 09:21:42.072999954,1634.024264,19.500736,81.701213,81.701213,2021-11-24 02:30:00,2021-11-24 03:51:42.072999954
2,Route3,9,2021-11-24 08:00:00,2021-11-24 10:26:57.714999914,2939.238235,31.652198,146.961912,146.961912,2021-11-24 02:30:00,2021-11-24 04:56:57.714999914
3,Route5,9,2021-11-24 08:00:00,2021-11-24 08:51:11.542999983,1023.847758,10.330118,51.192388,51.192388,2021-11-24 02:30:00,2021-11-24 03:21:11.542999983
4,Route7,7,2021-11-24 08:00:00,2021-11-24 08:56:47.556999922,1135.852291,13.297972,56.792615,56.792615,2021-11-24 02:30:00,2021-11-24 03:26:47.556999922


In [19]:
out_routes_df.to_csv('home/Routes_details.csv', index=False)    #to download the csv file

FileLinks ('home')

home/
  .nb_auth_file
  routes.csv
  depots.csv
  Routes_fset.json
  orders.csv
  customers.csv
  stores.csv
  testroutes.csv
  routes_med.json
  stores_med.csv
  routes_med.csv
  Routes_details.csv
  Stops_details.csv
home/pittsburgh_restaurants.gdb/
  a00000001.gdbindexes
  a00000001.gdbtable
  a00000001.gdbtablx
  a00000001.TablesByName.atx
  a00000002.gdbtable
  a00000002.gdbtablx
  a00000003.gdbindexes
  a00000003.gdbtable
  a00000003.gdbtablx
  a00000004.CatItemsByPhysicalName.atx
  a00000004.CatItemsByType.atx
  a00000004.FDO_UUID.atx
  a00000004.gdbindexes
  a00000004.gdbtable
  a00000004.gdbtablx
  a00000004.spx
  a00000005.CatItemTypesByName.atx
  a00000005.CatItemTypesByParentTypeID.atx
  a00000005.CatItemTypesByUUID.atx
  a00000005.gdbindexes
  a00000005.gdbtable
  a00000005.gdbtablx
  a00000006.CatRelsByDestinationID.atx
  a00000006.CatRelsByOriginID.atx
  a00000006.CatRelsByType.atx
  a00000006.FDO_UUID.atx
  a00000006.gdbindexes
  a00000006.gdbtable
  a00000006.gdbtablx
  a00000007.CatRelTypesByBackwardLabel.atx
  a00000007.CatRelTypesByDestItemTypeID.atx
  a00000007.CatRelTypesByForwardLabel.atx
  a00000007.CatRelTypesByName.atx
  a00000007.CatRelTypesByOriginItemTypeID.atx
  a00000007.CatRelTypesByUUID.atx
  a00000007.gdbindexes
  a00000007.gdbtable
  a00000007.gdbtablx
  gdb
  timestamps
  _gdb.0b787b2bb5bf48a.9.8.sr.lock
  a00000004.horizon
  a00000009.gdbtable
  a00000009.gdbtablx
  a00000009.gdbindexes
  a00000009.horizon
  a00000009.spx
  a00000001.freelist
  a0000000b.gdbtable
  a0000000b.gdbtablx
  a0000000b.gdbindexes
  a0000000b.horizon
  a0000000b.spx
  a0000000c.gdbtable
  a0000000c.gdbtablx
  a0000000c.gdbindexes
  a0000000c.horizon
  a00000004.freelist
  a00000006.freelist
  a0000000c.spx
  a0000000d.gdbtable
  a0000000d.gdbtablx
  a0000000d.gdbindexes
  a0000000d.horizon
  a0000000d.spx

In [20]:
out_stops_df = results.out_stops.sdf
out_stops_df[['Name','RouteName','Sequence','ArriveTime','DepartTime']].sort_values(by=['RouteName',
                                                                                       'Sequence'])

,Name,RouteName,Sequence,ArriveTime,DepartTime
40,Sheth Medical & General Store,Route1,1,2021-11-24 08:00:00.000000000,2021-11-24 08:00:00.000000000
39,Location 40,Route1,2,2021-11-24 08:05:23.861999989,2021-11-24 08:05:23.861999989
8,Location 9,Route1,3,2021-11-24 08:13:00.239000082,2021-11-24 08:13:00.239000082
37,Location 38,Route1,4,2021-11-24 08:19:55.713999987,2021-11-24 08:19:55.713999987
9,Location 10,Route1,5,2021-11-24 08:24:39.727999926,2021-11-24 08:24:39.727999926
35,Location 36,Route1,6,2021-11-24 08:34:23.125999928,2021-11-24 08:34:23.125999928
36,Location 37,Route1,7,2021-11-24 08:37:37.372999907,2021-11-24 08:37:37.372999907
10,Location 11,Route1,8,2021-11-24 08:53:07.697000027,2021-11-24 08:53:07.697000027
41,Sheth Medical & General Store,Route1,9,2021-11-24 09:05:51.737999916,2021-11-24 09:05:51.737999916
42,Ashok Medical & General Store,Route2,1,2021-11-24 08:00:00.000000000,2021-11-24 08:00:00.000000000


In [21]:
out_stops_df.to_csv('home/Stops_details.csv', index=False)    #to download the csv file

FileLinks ('home')

home/
  .nb_auth_file
  routes.csv
  depots.csv
  Routes_fset.json
  orders.csv
  customers.csv
  stores.csv
  testroutes.csv
  routes_med.json
  stores_med.csv
  routes_med.csv
  Routes_details.csv
  Stops_details.csv
home/pittsburgh_restaurants.gdb/
  a00000001.gdbindexes
  a00000001.gdbtable
  a00000001.gdbtablx
  a00000001.TablesByName.atx
  a00000002.gdbtable
  a00000002.gdbtablx
  a00000003.gdbindexes
  a00000003.gdbtable
  a00000003.gdbtablx
  a00000004.CatItemsByPhysicalName.atx
  a00000004.CatItemsByType.atx
  a00000004.FDO_UUID.atx
  a00000004.gdbindexes
  a00000004.gdbtable
  a00000004.gdbtablx
  a00000004.spx
  a00000005.CatItemTypesByName.atx
  a00000005.CatItemTypesByParentTypeID.atx
  a00000005.CatItemTypesByUUID.atx
  a00000005.gdbindexes
  a00000005.gdbtable
  a00000005.gdbtablx
  a00000006.CatRelsByDestinationID.atx
  a00000006.CatRelsByOriginID.atx
  a00000006.CatRelsByType.atx
  a00000006.FDO_UUID.atx
  a00000006.gdbindexes
  a00000006.gdbtable
  a00000006.gdbtablx
  a00000007.CatRelTypesByBackwardLabel.atx
  a00000007.CatRelTypesByDestItemTypeID.atx
  a00000007.CatRelTypesByForwardLabel.atx
  a00000007.CatRelTypesByName.atx
  a00000007.CatRelTypesByOriginItemTypeID.atx
  a00000007.CatRelTypesByUUID.atx
  a00000007.gdbindexes
  a00000007.gdbtable
  a00000007.gdbtablx
  gdb
  timestamps
  _gdb.0b787b2bb5bf48a.9.8.sr.lock
  a00000004.horizon
  a00000009.gdbtable
  a00000009.gdbtablx
  a00000009.gdbindexes
  a00000009.horizon
  a00000009.spx
  a00000001.freelist
  a0000000b.gdbtable
  a0000000b.gdbtablx
  a0000000b.gdbindexes
  a0000000b.horizon
  a0000000b.spx
  a0000000c.gdbtable
  a0000000c.gdbtablx
  a0000000c.gdbindexes
  a0000000c.horizon
  a00000004.freelist
  a00000006.freelist
  a0000000c.spx
  a0000000d.gdbtable
  a0000000d.gdbtablx
  a0000000d.gdbindexes
  a0000000d.horizon
  a0000000d.spx

In [22]:
# Create a map instance to visualize outputs in map
map_view_outputs = my_gis.map('Mumbai, Maharashtra, IND', zoomlevel=9.5)
map_view_outputs

MapView(layout=Layout(height='400px', width='100%'))

In [23]:
#Visusalize the inputsn with different symbols
map_view_outputs.draw(customers_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSCircle",
                                           "color": [76,115,0,255],"size": 8})
map_view_outputs.draw(stores_med_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSSquare",
                                           "color": [255,115,0,255], "size": 10})

#Visualize the first route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[0])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color":[0,100,240,255],"size":10})


#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[1])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [255,0,0,255],"size":10})
#Visualize the third route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[2])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [180,40,160,255],"size":10})
#Visualize the fifth route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[3])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [20,40,80,255],"size":10})
#Visualize the seventh route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[4])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [80,50,30,255],"size":10})

In [24]:
route_data = results.out_route_data.download('.')
route_data

'./_ags_rda442642ef5374616a75844a51e0c6e59.zip'

In [25]:
route_data_item = my_gis.content.add({"type": "File Geodatabase"}, route_data)
route_data_item

<Item title:"_ags_rda442642ef5374616a75844a51e0c6e59" type:File Geodatabase owner:Mainak_CSRE_User>

In [26]:
route_layers = arcgis.features.analysis.create_route_layers(route_data_item, 
                                                            delete_route_data_item=True)
for route_layer in route_layers:
    route_layer.share(org=True)
    display(route_layer.homepage)
    display(route_layer)

'https://www.arcgis.com/home/item.html?id=7083380ee3a74721a887ef0cf3aa1d1e'

<Item title:"Route1" type:Feature Collection owner:Mainak_CSRE_User>

'https://www.arcgis.com/home/item.html?id=74a35bc8e7ce4687a3cfc89ab3b4822c'

<Item title:"Route2" type:Feature Collection owner:Mainak_CSRE_User>

'https://www.arcgis.com/home/item.html?id=a9bc6d49338c44bb8317aef084abd984'

<Item title:"Route3" type:Feature Collection owner:Mainak_CSRE_User>

'https://www.arcgis.com/home/item.html?id=da25295336d549e08b544ef3cb628e00'

<Item title:"Route5" type:Feature Collection owner:Mainak_CSRE_User>

'https://www.arcgis.com/home/item.html?id=51220340017e462c9c37bf8c81b3dfb2'

<Item title:"Route7" type:Feature Collection owner:Mainak_CSRE_User>